<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/CBF_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

In [ ]:
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4],
    'place_id': [101, 102, 103, 104, 105, 106, 107, 108],
    'place_name': ['Place A', 'Place B', 'Place C', 'Place D', 'Place E', 'Place F', 'Place G', 'Place H'],
    'description': [
        'beautiful garden view',
        'lake view near mountains',
        'urban center shopping district',
        'quiet woodland area',
        'historic town center',
        'lively night district',
        'peaceful seaside area',
        'scenic rural landscape'
    ],
    'price': [120, 150, 200, 90, 80, 220, 130, 110],
    'rating1': [5, 4, 5, 2, 3, 4, 1, 2],
    'rating2': [3, 3, 4, 1, 2, 5, 1, 2]
}

df = pd.DataFrame(data)
# Menggabungkan rating menjadi satu nilai rata-rata
df['average_rating'] = df[['rating1', 'rating2']].mean(axis=1)


In [ ]:
# Menghitung TF-IDF dari deskripsi
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])

# Menambahkan kolom 'price' sebagai fitur tambahan
features = np.hstack([tfidf_matrix.toarray(), df['price'].values.reshape(-1, 1)])

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(features, df['average_rating'], test_size=0.25, random_state=42)

# Membuat dan melatih model
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [ ]:
# Membuat prediksi
predictions = model.predict(X_test)

# Menghitung MAE dan RMSE
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Mencetak hasil evaluasi
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")


MAE: 0.060063612838673786
RMSE: 0.06010845073002672


In [ ]:
# Mengubah prediksi dan nilai aktual menjadi kategori untuk confusion matrix
actual_categories = np.round(y_test).astype(int)
predicted_categories = np.round(predictions).astype(int)

# Confusion matrix
cm = confusion_matrix(actual_categories, predicted_categories, labels=np.unique(actual_categories))
print("Confusion Matrix:\n", cm)


Confusion Matrix:
 [[1]]
